# 1. optimizer 的基本情况

PyTorch 中的优化器是用于**管理并更新**模型中可学习参数的值，使得模型输出值更加接近真实标签值。

PyTorch 中提供了 Optimizer 类，定义如下：
```
class Optimizer(object):
    def __init__(self, params, defaults):        
        self.defaults = defaults
        self.state = defaultdict(dict)
        self.param_groups = []
```
主要属性：
- defaults：优化器的超参数
- state：参数的缓存，如 momentum 中需要用到前几次的梯度，就缓存在这个变量中
- param_groups：管理的参数组，是一个 list，参数可以划分为不同组，每组参数都是单独的字典，list 可以包含多个字典，实现对参数的差异化训练
- _step_count：记录更新次数，在学习率调整中使用

# 2. optimizer 的方法

## 2.1 step()

功能：执行一步梯度更新，执行 `optimizer.step()` 更新梯度，也就是 `weight.data` 减去梯度乘以学习率。

In [1]:
import torch

weight = torch.randn((2, 2), requires_grad=True)
weight.grad = torch.ones((2, 2))  # 梯度设为 1

optimizer = torch.optim.SGD([weight], lr=1)
print("weight before step:{}".format(weight.data))
optimizer.step()
print("weight after step:{}\n".format(weight.data))

weight before step:tensor([[-0.5174, -0.9269],
        [ 0.9223,  0.6238]])
weight after step:tensor([[-1.5174, -1.9269],
        [-0.0777, -0.3762]])



In [2]:
print("weight in optimizer:{}\nweight in weight:{}".format(id(optimizer.param_groups[0]['params'][0]), id(weight)))

weight in optimizer:2288419532288
weight in weight:2288419532288


可以看到优化器的 param_groups 中存储的参数和 weight 的内存地址是一样的，所以优化器中保存的是参数的地址索引，而不是把参数复制到优化器中。

## 2.2 zero_grad()

功能：清空所管理参数的梯度。由于 PyTorch 的特性是张量的梯度不自动清零，因此每次反向传播之后都需要清空梯度。

In [3]:
print("weight.grad is \n{}".format(weight.grad))
optimizer.zero_grad()
print("after optimizer.zero_grad(), weight.grad is {}\n".format(weight.grad))

weight.grad is 
tensor([[1., 1.],
        [1., 1.]])
after optimizer.zero_grad(), weight.grad is None



## 2.3 add_param_group()

功能：在 param_groups 这个 list 后面添加（append）参数组

In [4]:
print("optimizer.param_groups is\n{}\n".format(optimizer.param_groups))

w2 = torch.randn((3, 3), requires_grad=True)
optimizer.add_param_group({"params": w2, 'lr': 0.0001})

print("optimizer.param_groups is\n{}".format(optimizer.param_groups))

optimizer.param_groups is
[{'params': [tensor([[-1.5174, -1.9269],
        [-0.0777, -0.3762]], requires_grad=True)], 'lr': 1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}]

optimizer.param_groups is
[{'params': [tensor([[-1.5174, -1.9269],
        [-0.0777, -0.3762]], requires_grad=True)], 'lr': 1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}, {'params': [tensor([[-0.8968, -0.7357, -1.8949],
        [-0.8327,  1.5896, -1.0585],
        [ 1.0978, -0.6106,  0.2272]], requires_grad=True)], 'lr': 0.0001, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}]


## 2.4 state_dict()

功能：获取优化器当前状态信息字典

In [5]:
optimizer = torch.optim.SGD([weight], lr=0.1, momentum=0.9)
print("state_dict:\n", optimizer.state_dict())

# 使用 torch.save() 把 state_dict 保存到 pkl 文件中
torch.save(optimizer.state_dict(), "./optimizer_state_dict.pkl")

state_dict:
 {'state': {}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0]}]}


## 2.5 load_state_dict()

功能：加载保存的优化器状态信息字典，主要用于模型的断点续训练。我们可以在每隔一定数量的 epoch 就保存模型的 state_dict 到硬盘，在意外终止训练时，可以继续加载上次保存的状态，继续训练。

In [6]:
# 保存了 state_dict 之后，使用 torch.load() 把 state_dict 加载到内存中
state_dict = torch.load("./optimizer_state_dict.pkl")

# 重新构建优化器，并使用 load_state_dict() 将保存的 state_dict 加载到模型中
optimizer = torch.optim.SGD([weight], lr=0.2, momentum=0.9)  # 这里设置的参数在加载后会被覆盖
optimizer.load_state_dict(state_dict)
print("state_dict after load state:\n", optimizer.state_dict())

state_dict after load state:
 {'state': {}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0]}]}


C:\Users\24606\AppData\Local\Temp\ipykernel_20500\887406690.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("./optimizer_state_dict.pkl")


# 3. optimizer 的参数

## 3.1 学习率

初始化时，一般把学习率设置为比较小的数，如 0.01，0.001。

## 3.2 momentum 动量

momentum（动量、冲量）：结合当前梯度与上一次更新信息，用于当前更新。

**常规的更新参数（梯度下降）：**

$w_{i+1}=w_{i}-l r * g\left(w_{i}\right)$

**在 PyTroch 中更新参数（梯度下降）时，不仅考虑当前的梯度，还会结合前面的梯度。参数的更新公式是：**

$w_{i+1}=w_{i}-lr * v_{i}$

$v_{i}=m * v_{i-1}+g\left(w_{i}\right)=···=m^3* v_{i-3}+m^2 * g\left(w_{i-2}\right)+m * g\left(w_{i-1}\right)+g\left(w_{i}\right)=···$ 

其中 $w_{i+1}$ 表示第 $i+1$ 次更新的参数，lr 表示学习率，$v_{i}$ 表示更新量，$m$ 表示 momentum 系数，$g(w_{i})$ 表示 $w_{i}$ 的梯度。

momentum 系数 $m$ 可以理解为记忆周期系数，$m$ 越小，记忆周期越短，$m$ 越大，记忆周期越长。通常将 $m$ 设置为 0.9，那么 $\frac{1}{1-m}=\frac{1}{1-0.9}=10$，表示更关注最近 10 次参数更新时的梯度。

## 3.3 weight_decay 权值衰减（L2 正则项）

**正则项有 L1 和 L2 正则项两种：**
- L1 正则项：$\sum\boldsymbol|{w}_{i}|$，使用 L1 正则项会产生稀疏参数值（有 0 值的出现）
- L2 正则项：$\sum\boldsymbol{w}_{i}^{2}$， L2 正则项又被称为权值衰减(weight decay)


# 4. PyTroch 提供的 10 种优化器

**torch.optim.SGD(params, lr, momentum=0, dampening=0, weight_decay=0, nesterov=False)**

随机梯度下降法

主要参数：
- params：管理的参数组，是一个 list，参数可以划分为不同组，每组参数都是单独的字典
- lr：初始学习率
- momentum：动量系数
- weight_decay：L2 正则化系数
- nesterov：是否采用 NAG 梯度下降方法，默认为 False，一般不采用

**其他：**
- optim.Adagrad：自适应学习率梯度下降法
- optim.RMSprop：Adagrad 的改进
- optim.Adadelta
- optim.Adam：RMSProp 集合 Momentum，这个是目前最常用的优化器，因为它可以使用较大的初始学习率。
- optim.Adamax：Adam 增加学习率上限
- optim.SparseAdam：稀疏版的 Adam
- optim.ASGD：随机平均梯度下降
- optim.Rprop：弹性反向传播，这种优化器通常是在所有样本都一起训练，也就是 batchsize 为全部样本时使用。
- optim.LBFGS：BFGS 在内存上的改进